# 3. Agent Routing / Supervisor with Semantic Kernel

In this pattern, a supervisor agent routes user queries to specialized agents based on the content of the request. Semantic Kernel makes this pattern easy to implement using multiple specialized agents coordinated by a router agent.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from typing import Annotated, Literal
import json

Set up the kernel and service

In [ ]:
# Create a kernel instance
kernel = Kernel()

# Add Azure OpenAI chat completion service
service_id = "azure_openai"
kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
        deployment_name="gpt-4.1-mini",
    )
)

Create specialized plugins and agents

In [ ]:
class WeatherPlugin:
    """
    A plugin that provides weather information for specific cities.
    """
    
    @kernel_function(
        description="Get the current weather for a specified location",
        name="get_weather"
    )
    def get_weather(
        self,
        location: Annotated[str, "The name of the city to get the weather for. Must be one of 'Chicago', 'New York', or 'Los Angeles'"]
    ) -> Annotated[str, "A string describing the current weather in the specified location"]:
        """
        Get the current weather for a specified location.
        """
        weather_data = {
            "New York": "Sunny, 25°C",
            "Los Angeles": "Cloudy, 22°C",
            "Chicago": "Rainy, 18°C"
        }
        return weather_data.get(location, "Weather data not available for this location.")

# Create specialized kernels for different agents
weather_kernel = Kernel()
weather_kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
        deployment_name="gpt-4.1-mini",
    )
)
weather_kernel.add_plugin(WeatherPlugin(), plugin_name="weather")

Create travel advisor agent

In [ ]:
# Create travel advisor kernel (no special plugins needed)
travel_kernel = Kernel()
travel_kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
        deployment_name="gpt-4.1-mini",
    )
)

Create specialized agents

In [ ]:
# Weather Agent
weather_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=weather_kernel,
    name="WeatherAgent",
    instructions="You are a weather specialist. Use the weather tools to provide accurate weather information for supported cities.",
    execution_settings={
        service_id: weather_kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)
weather_agent.execution_settings[service_id].function_choice_behavior = FunctionChoiceBehavior.Auto()

# Travel Advisor Agent
travel_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=travel_kernel,
    name="TravelAgent",
    instructions="You are a travel expert. Based on the user's input, suggest 1 travel destination with helpful details.",
    execution_settings={
        service_id: travel_kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)

# Router Agent
router_agent = ChatCompletionAgent(
    service_id=service_id,
    kernel=kernel,
    name="RouterAgent",
    instructions="""You are a router that determines which specialist to use for user queries.
    
Respond with ONLY a JSON object in this exact format:
{"agent": "weather"} - for weather-related questions
{"agent": "travel"} - for travel advice questions  
{"agent": "unknown"} - if you're unsure or the query doesn't fit

Do not include any other text, explanations, or formatting.""",
    execution_settings={
        service_id: kernel.get_prompt_execution_settings_from_service_id(service_id)
    }
)

Create router function to determine which agent to use

In [ ]:
async def route_query(user_message: str) -> str:
    """
    Route user query to appropriate agent
    """
    chat_history = ChatHistory()
    chat_history.add_user_message(user_message)
    
    # Get routing decision
    async for response in router_agent.invoke(chat_history):
        try:
            # Parse the JSON response
            routing_result = json.loads(str(response.content))
            agent_choice = routing_result.get("agent", "unknown")
            print(f"Router decision: {agent_choice}")
            return agent_choice
        except json.JSONDecodeError:
            print(f"Failed to parse router response: {response.content}")
            return "unknown"
        break
    return "unknown"

Create supervisor function to coordinate agents

In [ ]:
async def supervisor_agent(user_message: str) -> str:
    """
    Supervisor that routes queries to appropriate specialized agents
    """
    # Route the query
    agent_choice = await route_query(user_message)
    
    # Create chat history for the chosen agent
    chat_history = ChatHistory()
    chat_history.add_user_message(user_message)
    
    if agent_choice == "weather":
        print("→ Routing to Weather Agent")
        async for response in weather_agent.invoke(chat_history):
            return str(response.content)
            
    elif agent_choice == "travel":
        print("→ Routing to Travel Agent")
        async for response in travel_agent.invoke(chat_history):
            return str(response.content)
            
    else:
        print("→ Query type unknown")
        return ("I can not assist with that request. "
               "Could you please rephrase or ask something else? "
               "I can help you with travel advice or weather information.")
    
    return "Sorry, I couldn't process your request."

Test the routing system

In [ ]:
# Test weather routing
weather_query = "What's the weather like in New York?"
print(f"User: {weather_query}")
response = await supervisor_agent(weather_query)
print(f"Assistant: {response}\n")

In [ ]:
# Test travel routing
travel_query = "I want to visit a place with cats"
print(f"User: {travel_query}")
response = await supervisor_agent(travel_query)
print(f"Assistant: {response}\n")

In [ ]:
# Test unknown routing
unknown_query = "What's 2 + 2?"
print(f"User: {unknown_query}")
response = await supervisor_agent(unknown_query)
print(f"Assistant: {response}\n")

## Multi-turn conversation example

In [ ]:
# Simulate a multi-turn conversation
queries = [
    "What can you tell me about the weather in Mumbai?",
    "How about Chicago weather?",
    "I want to travel somewhere warm and sunny"
]

print("=== Multi-turn Conversation ===")
for query in queries:
    print(f"\nUser: {query}")
    response = await supervisor_agent(query)
    print(f"Assistant: {response}")

## Key Differences from LangChain Routing

1. **Agent-Based Architecture**: Semantic Kernel uses `ChatCompletionAgent` objects as first-class citizens, each with their own specialized kernels and capabilities.

2. **Simplified Routing Logic**: Instead of complex graph structures, we use a simple async function to coordinate between agents.

3. **Service Isolation**: Each agent has its own kernel with specific services and plugins, providing better separation of concerns.

4. **Cleaner State Management**: No need for complex state graphs - each agent manages its own conversation state.

5. **Flexible Coordination**: The supervisor function can easily be extended to handle more complex routing logic, multi-agent collaboration, or conditional workflows.

6. **Natural Language Routing**: The router agent uses natural language understanding rather than rigid rule-based routing.

7. **Plugin Ecosystem**: Each specialized agent can have its own set of plugins, making it easy to extend capabilities without affecting other agents.

This approach provides better modularity and makes it easier to add new specialized agents or modify existing ones without affecting the overall system architecture.

### Benefits:

- **Scalability**: Easy to add new specialized agents
- **Maintainability**: Each agent is self-contained
- **Flexibility**: Simple to modify routing logic
- **Testability**: Each agent can be tested independently
- **Performance**: Agents only load the capabilities they need"